In [19]:
from PIL import Image
import os, glob, numpy as np
from sklearn.model_selection import train_test_split

caltech_dir = "./train/"
categories = ["1", "2", "3", "4", "5", "6", "7"]
nb_classes = len(categories)

image_w = 64
image_h = 64

pixels = image_h * image_w * 3

X = []
y = []

for idx, cat in enumerate(categories):
    
    #one-hot 돌리기.
    label = [0 for i in range(nb_classes)]
    label[idx] = 1

    image_dir = caltech_dir + "/" + cat
    files = glob.glob(image_dir+"/*.jpg")
    print(cat, " 파일 길이 : ", len(files))
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)

        X.append(data)
        y.append(label)

        if i % 700 == 0:
            print(cat, " : ", f)

X = np.array(X)
y = np.array(y)


X_train, X_test, y_train, y_test = train_test_split(X, y)
xy = (X_train, X_test, y_train, y_test)
np.save("./numpy_data/multi_image_data.npy", xy)

print("ok", len(y))

1  파일 길이 :  7512
1  :  ./train//1\9_0.jpg
1  :  ./train//1\tri_0_1470.jpg
1  :  ./train//1\tri_0_2326.jpg
1  :  ./train//1\tri_0_3217.jpg
1  :  ./train//1\tri_0_4082.jpg
1  :  ./train//1\tri_0_4958.jpg
1  :  ./train//1\tri_0_5837.jpg
1  :  ./train//1\tri_0_6721.jpg
1  :  ./train//1\tri_0_7574.jpg
1  :  ./train//1\tri_0_8467.jpg
1  :  ./train//1\tri_0_9351.jpg
2  파일 길이 :  7392
2  :  ./train//2\8_0.jpg
2  :  ./train//2\tri_0_1755.jpg
2  :  ./train//2\tri_0_261.jpg
2  :  ./train//2\tri_0_3493.jpg
2  :  ./train//2\tri_0_4350.jpg
2  :  ./train//2\tri_0_5210.jpg
2  :  ./train//2\tri_0_6072.jpg
2  :  ./train//2\tri_0_6937.jpg
2  :  ./train//2\tri_0_7817.jpg
2  :  ./train//2\tri_0_8673.jpg
2  :  ./train//2\tri_0_9539.jpg
3  파일 길이 :  7508
3  :  ./train//3\3_0.jpg
3  :  ./train//3\tri_0_169.jpg
3  :  ./train//3\tri_0_251.jpg
3  :  ./train//3\tri_0_3361.jpg
3  :  ./train//3\tri_0_4194.jpg
3  :  ./train//3\tri_0_5061.jpg
3  :  ./train//3\tri_0_5920.jpg
3  :  ./train//3\tri_0_6795.jpg
3  :  ./train

In [21]:
import os, glob, numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import keras.backend.tensorflow_backend as K

X_train, X_test, y_train, y_test = np.load('./numpy_data/multi_image_data.npy', allow_pickle=True)
print(X_train.shape)
print(X_train.shape[0])

(39365, 64, 64, 3)
39365


In [22]:
categories = ["1", "2", "3", "4", "5", "6", "7"]
nb_classes = len(categories)

#일반화
X_train = X_train.astype(float) / 255
X_test = X_test.astype(float) / 255

In [23]:
model = Sequential()
model.add(Conv2D(32, (3,3), padding="same", input_shape=X_train.shape[1:], activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_dir = './model'

if not os.path.exists(model_dir):
    os.mkdir(model_dir)

model_path = model_dir + '/multi_img_classification.model'
checkpoint = ModelCheckpoint(filepath=model_path , monitor='val_loss', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=6)

In [24]:
history = model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test),validation_split=0.2, callbacks=[checkpoint, early_stopping])

Train on 39365 samples, validate on 13122 samples
Epoch 1/50
39365/39365 [==============================] - 248s 6ms/step - loss: 1.6114 - accuracy: 0.3416 - val_loss: 1.2972 - val_accuracy: 0.4969

Epoch 00001: val_loss improved from inf to 1.29719, saving model to ./model/multi_img_classification.model
Epoch 2/50
39365/39365 [==============================] - 263s 7ms/step - loss: 1.2816 - accuracy: 0.4867 - val_loss: 1.0656 - val_accuracy: 0.6067

Epoch 00002: val_loss improved from 1.29719 to 1.06559, saving model to ./model/multi_img_classification.model
Epoch 3/50
39365/39365 [==============================] - 272s 7ms/step - loss: 1.0935 - accuracy: 0.5700 - val_loss: 0.9573 - val_accuracy: 0.6466

Epoch 00003: val_loss improved from 1.06559 to 0.95734, saving model to ./model/multi_img_classification.model
Epoch 4/50
39365/39365 [==============================] - 427s 11ms/step - loss: 0.9677 - accuracy: 0.6238 - val_loss: 0.7897 - val_accuracy: 0.7206

Epoch 00004: val_loss im

In [8]:
from PIL import Image
import os, glob, numpy as np
from sklearn.model_selection import train_test_split
import os, glob, numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import keras.backend.tensorflow_backend as K
import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
model_dir = './model'

model_path = model_dir + '/multi_img_classification.model'
model_path

'./model/multi_img_classification.model'

In [9]:
model = keras.models.load_model(model_path)
model

In [10]:
model.predict(load_img("../frontend/gg.png"))

AttributeError: module 'keras' has no attribute 'load_img'

In [12]:
import numpy as np
import os
# 랜덤시드 고정시키기
np.random.seed(5)

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# 데이터셋 불러오기
data_aug_gen = ImageDataGenerator(rescale=1./255, 
                                  rotation_range=15,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  shear_range=0.5,
                                  zoom_range=[0.8, 2.0],
                                  horizontal_flip=True,
                                  vertical_flip=True,
                                  fill_mode='nearest')

for idx in range(1,8):
    categori = f"./train/{idx}/"
    for catimg in os.listdir(categori):
        if len(os.listdir(categori)) > 7500:
            break
        img = load_img(categori+"/"+catimg)
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)

        i = 0

        # 이 for는 무한으로 반복되기 때문에 우리가 원하는 반복횟수를 지정하여, 지정된 반복횟수가 되면 빠져나오도록 해야합니다.
        for batch in data_aug_gen.flow(x, batch_size=1, save_to_dir=categori, save_prefix='tri', save_format='jpg'):
            i += 1
            if i > 120: 
                break

7512
7392
7508
7503
7509
7525
7538
